<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/EDA12_1%EC%B0%A8_%EC%B5%9C%EC%A2%85_%EB%B3%80%EC%88%98%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
path = './gdrive/My Drive/빅콘테스트2020/데이터/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/'

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀투수.csv', encoding='utf-8-sig')
pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인투수.csv', encoding='utf-8-sig')
team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀타자.csv', encoding='utf-8-sig')
hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인타자.csv', encoding='utf-8-sig')
team_data = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig')

In [4]:
data_list = [team_pitcher, pitcher, team_hitter, hitter,team_data]

In [5]:
for data in data_list:
  data['일자'] = pd.to_datetime(data['일자'],format='%Y-%m-%d', errors='raise')

In [56]:
## 저장 공간
team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀투수.csv', encoding='utf-8-sig', index=False)
pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인투수.csv', encoding='utf-8-sig', index=False)
team_data.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig', index=False)
team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀타자.csv', encoding='utf-8-sig', index=False)
hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인타자.csv', encoding='utf-8-sig', index=False)

In [ ]:
# 변화 확인하기
team_pitcher.columns

Index(['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '평균자책점', 'RA/9', 'ERC', 'FIP', 'LOB',
       '홈런율', '삼진율', 'K9', 'BB9', 'HR9', 'KBB', 'whip', 'P/G', 'P/IP', 'P/PA',
       'PFR', 'CYP', 'F-E', 'R-E', '총합연봉', '평균연봉', 'OAVG', 'OOBP', '단타',
       'OSLG', 'OOPS', 'kwERA', 'K%', 'BB%'],
      dtype='object')

OOPS수정

In [ ]:
team_pitcher['OOPS'] = team_pitcher['OOBP'] + team_pitcher['OSLG']
pitcher['OOPS'] = pitcher['OOBP'] + pitcher['OSLG']

소수점 3자리로통일

In [ ]:
team_pitcher = team_pitcher.round(3)
pitcher = pitcher.round(3)
team_hitter = team_hitter.round(3)
hitter = hitter.round(3)

공통 데이터 구하기

In [ ]:
team_data.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig', index=False)

In [ ]:
team_code = ['NC', 'OB', 'WO', 'HT', "LG", 'SS', 'KT', 'LT', 'SK', "HH"]
def createDate(row):
  previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['시즌'] == row['시즌']) & (team_pitcher['일자'] < row['일자']) ].tail(1)
  if len(previous) == 0:
    row['경기간간격'] = 0
    return row
  previous = previous.sum()
  a = (row['일자'] - previous['일자'])
  # print(a)
  row['경기간간격'] = a.days
  # print(row['일자'],  previous['일자'], row['경기간간격'])
  return row

In [ ]:
team_data = team_data.apply(createDate, axis=1)

In [ ]:
team_data['월'] =  team_data['일자'].dt.month

In [ ]:

def team_encoding(row, team_code):
  if row['팀코드'] == team_code:
    row[team_code] = 1
  else:
    row[team_code] = 0
  return row

def createHome(row):
  if row['초말'] == 'B':
    row['초말'] = 1
  else:
    row['초말'] = 0
  return row

def createOutIn(row):
  if row['이닝*3'] < 24:
    row['우취'] = 1
    row['연장'] = 0
  elif row['이닝*3'] > 27:
    row['우취'] = 0
    row['연장'] = 1
  else:
    row['우취'] = 0
    row['연장'] = 0
  return row


In [ ]:
team_code = ['NC', 'OB', 'WO', 'HT', "LG", 'SS', 'KT', 'LT', 'SK', "HH"]

for team in team_code:
  team_data = team_data.apply(team_encoding, axis=1, team_code=team)


team_data = team_data.apply(createHome, axis=1)
team_data = team_data.apply(createOutIn, axis=1)

In [ ]:
# 함수 모아놓은 곳
def gainCreate(row):
    row['득점'] = int(team_data[(row['게임키'] == team_data['게임키']) & (team_data['팀코드'] == row['상대팀코드'])]['실점'])
    return row

def createInfo(row):
    previous = team_data[(team_data['팀코드'] == row['팀코드']) & (team_data['일자'] < row['일자']) & (team_data['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전총득점'] = 0
        row['시작전총실점'] = 0
        row['시작전평균득점'] = 0
        row['시작전평균실점'] = 0
    else:
        gainSum = previous['득점'].sum()
        gainMean = gainSum / len(previous)
        lossSum = previous['실점'].sum()
        lossMean = lossSum / len(previous)
        row['시작전총득점'] = gainSum
        row['시작전총실점'] = lossSum
        row['시작전평균득점'] = gainMean
        row['시작전평균실점'] = lossMean
    return row

def winP(row):
    previous = team_data[(team_data['팀코드'] == row['팀코드']) & (team_data['일자'] < row['일자']) & (team_data['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전승수'] = 0
        row['시작전패배수'] = 0
        row['시작전승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['시작전승수'] = winSum
        row['시작전패배수'] = lossSum
        row['시작전승률'] = winMean
    return row

In [ ]:
team_data = team_data.apply(gainCreate, axis=1)
team_data = team_data.apply(createInfo, axis=1)
team_data = team_data.apply(winP, axis=1)


In [ ]:
team_data['피타고리안기대승률'] = (team_data['시작전총득점'] ** 2) / ((team_data['시작전총득점'] ** 2) + (team_data['시작전총실점'] ** 2))

In [9]:
team_data = team_data.fillna(0)

상대전적별 데이터 구하기

In [13]:
vs_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA7-개인타자_상대별_세이버매트릭스_합본.csv', encoding='utf-8-sig')
vs_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA7-팀타자_상대별_세이버매트릭스_합본.csv', encoding='utf-8-sig')
vs_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_개안투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig')
vs_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_팀투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig')
# team_data = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig')

In [93]:
vs_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀투수.csv', encoding='utf-8-sig', index=False)
vs_team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀타자.csv', encoding='utf-8-sig', index=False)

In [14]:
vs_team_hitter

,Unnamed: 0,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,상대출루율,상대장타력,상대ops,상대타율,상대ISO,상대BABIP,상대BB/K,상대K%,상대BB%,상대wOBA,상대wRAA,상대wRC,상대HR%,상대GPA
0,0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,635,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.299663,0.314607,0.614270,0.228464,0.086142,0.263636,0.565217,0.154362,0.087248,0.288293,-14.656294,8.018131,0.010067,0.213500
6396,636,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.340000,0.389513,0.729513,0.265918,0.123596,0.303167,0.681818,0.144737,0.095395,0.333920,-2.931033,20.276013,0.013158,0.250378
6397,637,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.363934,0.433213,0.797147,0.314079,0.119134,0.360870,0.382979,0.152104,0.058252,0.362672,4.736070,28.247471,0.012945,0.272074
6398,638,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.306931,0.365942,0.672873,0.246377,0.119565,0.287081,0.377049,0.200000,0.075410,0.307934,-9.804971,13.402075,0.026230,0.229604


In [59]:
def createRC(row):
    previous = vs_vs_vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드']) & (vs_team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['RC'] = 0
        return row
    previous_sum = previous.sum()
    upA = previous_sum['안타']+previous_sum['4구']-previous_sum['도루실패']+previous_sum['사구']-previous_sum['병살타']
    upB = (previous_sum['단타']*1.125)+(1.69*previous_sum['2루타'])+(3.02*previous_sum['3루타'])+(3.73*previous_sum['홈런'])+ 0.29*(previous_sum['4구']-previous_sum['고4']+previous_sum['사구'])
    down = previous_sum['타수']+previous_sum['4구']+previous_sum['사구']+previous_sum['희비']+previous_sum['희타']
    if down == 0:
        row['RC'] = 0
        return row
    row['RC'] = ((2.4*down+upA)*(3*down+upB)/(9*down)) - (0.9*down)
    
    return row

def createRC27(row):
    previous = vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드']) & (vs_team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['RC27'] = 0
        return row
    previous_sum = previous.sum()
    down = previous_sum['타수'] - previous_sum['안타']+previous_sum['병살타']+previous_sum['도루실패']+previous_sum['희타']+previous_sum['희비']
    if down == 0:
        row['RC27'] = 0
        return row
    row['RC27'] = (row['RC']/down)*27
    
    return row


def createSPD(row):
    previous = vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드']) & (vs_team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['SPD'] = 0
        return row
    previous_sum = previous.sum()
    try:
        f1 = ((previous_sum['도루']+3)/(previous_sum['도루']+previous_sum['도루실패']+7)-0.4) * 20
        f2 = ((previous_sum['도루']+previous_sum['도루실패'])/(previous_sum['단타']+previous_sum['4구']+previous_sum['사구']))**0.5/0.07
        f3 = previous_sum['3루타']/(previous_sum['타수']-previous_sum['홈런']-previous_sum['삼진'])/ 0.02*10
        f4 = ((previous_sum['득점']-previous_sum['홈런'])/(previous_sum['안타']+previous_sum['4구']-previous_sum['홈런']-previous_sum['사구'])-0.1)/0.04
        f5 = (0.055-previous_sum['병살타']/(previous_sum['타수']-previous_sum['홈런']-previous_sum['삼진']))/0.005
    except ZeroDivisionError:
        row['SPD'] = 0
        return row
    row['SPD'] = (f1+f2+f3+f4+f5) / 5
    return row

def createXR(row):
    previous = vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드']) & (vs_team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['XR'] = 0
        return row
    previous_sum = previous.sum()
    try:
        Y1 = 0.5*previous_sum['단타']+0.72*previous_sum['2루타']+1.04*previous_sum['3루타']+1.44*previous_sum['홈런']
        Y2 = (previous_sum['사구']+previous_sum['4구']-previous_sum['고4'])*0.34+0.25*previous_sum['고4']+0.18*previous_sum['도루']-0.32*previous_sum['도루실패']
        Y3 = -(previous_sum['타수']-previous_sum['안타']-previous_sum['삼진'])*0.09-0.098*previous_sum['삼진']-0.37*previous_sum['병살타']+0.37*previous_sum['희비']+0.04*previous_sum['희타']
    except ZeroDivisionError:
        row['XR'] = 0
        return row
    row['XR'] = Y1+Y2+Y3
    return row

def createSecA(row):
    previous = vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['일자'] < row['일자']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드'])]
    if len(previous) == 0:
        row['SecA'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['4구']+(previous_sum['2루타']+previous_sum['3루타']*2+previous_sum['홈런']*3)+previous_sum['도루']-previous_sum['도루실패']
    down = previous_sum['타수']
    if down == 0:
        row['SecA'] = 0
        return row
    row['SecA'] = up/down
    return row

def createPSN(row):
    previous = vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['일자'] < row['일자']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드'])]
    if len(previous) == 0:
        row['PSN'] = 0
        return row
    previous_sum = previous.sum()
    try:
        psn = 2*(previous_sum["홈런"]*previous_sum["도루"])/(previous_sum["홈런"]+previous_sum["도루"])
    except ZeroDivisionError:
        psn = 0
    row['PSN']= psn
    return row

In [16]:
try:
  vs_team_hitter = vs_team_hitter.apply(createRC, axis=1)
  vs_team_hitter = vs_team_hitter.apply(createRC27, axis=1)
  vs_team_hitter = vs_team_hitter.apply(createSPD, axis=1)
  vs_team_hitter = vs_team_hitter.apply(createXR, axis=1)
except:
  vs_team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀타자.csv', encoding='utf-8-sig', index=False)
finally:
  vs_team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀타자.csv', encoding='utf-8-sig', index=False)

In [60]:
vs_team_hitter = vs_team_hitter.apply(createSecA, axis=1)
vs_team_hitter = vs_team_hitter.apply(createPSN, axis=1)

In [17]:
vs_team_hitter

,Unnamed: 0,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,상대출루율,상대장타력,상대ops,상대타율,상대ISO,상대BABIP,상대BB/K,상대K%,상대BB%,상대wOBA,상대wRAA,상대wRC,상대HR%,상대GPA,RC,RC27,SPD,XR
0,0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
1,1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
2,2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
3,3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
4,4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,635,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.299663,0.314607,0.614270,0.228464,0.086142,0.263636,0.565217,0.154362,0.087248,0.288293,-14.656294,8.018131,0.010067,0.213500,25.754819,3.249440,4.448517,27.252
6396,636,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.340000,0.389513,0.729513,0.265918,0.123596,0.303167,0.681818,0.144737,0.095395,0.333920,-2.931033,20.276013,0.013158,0.250378,34.422495,4.404774,3.027312,34.698
6397,637,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.363934,0.433213,0.797147,0.314079,0.119134,0.360870,0.382979,0.152104,0.058252,0.362672,4.736070,28.247471,0.012945,0.272074,40.375889,5.166583,3.683077,40.444
6398,638,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.306931,0.365942,0.672873,0.246377,0.119565,0.287081,0.377049,0.200000,0.075410,0.307934,-9.804971,13.402075,0.026230,0.229604,30.931506,3.796139,3.918947,31.132


In [61]:
def createPFR(row):
    previous = vs_vs_team_pitcher[(vs_vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['PFR'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = (previous_sum["삼진"]+previous_sum["4구"])/(previous_sum["이닝*3"]/3)
    except ZeroDivisionError:
        pg = 0
    row["PFR"] = pg
    return row
def createPG(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['P/G'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = previous_sum["투구수"]/previous.count()[0]
    except ZeroDivisionError:
        pg = 0
    row["P/G"] = pg
    return row

def createPIP(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['P/IP'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = previous_sum["투구수"]/(previous_sum["이닝*3"]/3)
    except ZeroDivisionError:
        pg = 0
    row["P/IP"] = pg
    return row
def createPPA(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['P/PA'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = previous_sum["투구수"]/previous_sum["타자"]
    except ZeroDivisionError:
        pg = 0
    row["P/PA"] = pg
    return row

def createCYP(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['CYP'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = (previous_sum["이닝*3"]/6-previous_sum["자책점"])+previous_sum["삼진"]/10+len(previous[previous['결과'] == 'W'])
    except ZeroDivisionError:
        pg = 0
    row["CYP"] = pg
    return row

def createkwERA(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드']) & (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['kwERA'] = 0
        return row
    previous_sum = previous.sum()
    try:
        up = 5.40-12*(previous_sum['삼진']-previous_sum['4구'])/previous_sum['타자']
    except ZeroDivisionError:
        row['kwERA'] = 0
        return row
    row['kwERA'] = up
    return row

def createK(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드']) & (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['K%'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['삼진']
    down = previous_sum['타자']
    if down == 0:
        row['K%'] = 0
        return row
    row['K%'] = up/down
    return row

def createBB(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드']) & (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['BB%'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['4구']
    down = previous_sum['타자']
    if down == 0:
        row['BB%'] = 0
        return row
    row['BB%'] = up/down
    return row

def createOAVG(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드']) & (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['OAVG'] = 0
        return row
    previous_sum = previous.sum()
    try:
        oavg = previous_sum["안타"]/previous_sum["타수"]
    except ZeroDivisionError:
        oavg = 0
    row['OAVG']= oavg
    return row

def createOOBP(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드']) & (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['OOBP'] = 0
        return row
    previous_sum = previous.sum()
    try:
        oobp = (previous_sum["안타"]+previous_sum["4구"]+previous_sum["사구"])/(previous_sum["타수"]+previous_sum["4구"]+previous_sum["사구"]+previous_sum["희비"])
    except ZeroDivisionError:
        oobp = 0
    row['OOBP']= oobp
    return row

def createOSLG(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드']) & (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['OSLG'] = 0
        return row
    previous_sum = previous.sum()
    try:
        oslg = (previous_sum["단타"]+2*previous_sum["2루타"]+3*previous_sum["3루타"]+4*previous_sum["홈런"])/previous_sum["타수"]
    except ZeroDivisionError:
        oslg = 0
    row['OSLG']= oslg
    return row

In [22]:
try:
  vs_team_pitcher = vs_team_pitcher.apply(createPFR, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createPG, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createPIP, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createPPA, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createCYP, axis=1)
  vs_team_pitcher['상대F-E'] = vs_team_pitcher['상대FIP'] - vs_team_pitcher['상대평균자책점']
  vs_team_pitcher['상대R-E'] = vs_team_pitcher['상대RA/9'] - vs_team_pitcher['상대평균자책점']
except:
  vs_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀투수.csv', encoding='utf-8-sig', index=False)
  print('오류발생')
finally:
  vs_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀투수.csv', encoding='utf-8-sig', index=False)

오류발생


In [63]:
try:
  vs_team_pitcher = vs_team_pitcher.apply(createkwERA, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createK, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createBB, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createOAVG, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createOOBP, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createOSLG, axis=1)
  vs_team_pitcher['OOPS'] = vs_team_pitcher['OSLG'] + vs_team_pitcher['OOBP']
except:
  vs_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀투수.csv', encoding='utf-8-sig', index=False)
  print('오류발생')
finally:
  vs_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀투수.csv', encoding='utf-8-sig', index=False)

In [64]:
vs_team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,상대평균자책점,상대RA/9,상대ERC,상대FIP,상대LOB,상대홈런율,상대삼진율,상대K9,상대BB9,상대HR9,상대KBB,상대whip,PFR,P/G,P/IP,P/PA,CYP,상대F-E,상대R-E,kwERA,K%,BB%,OAVG,OOBP,OSLG,OOPS
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000,2020,6.211268,6.211268,5.269202,5.498131,0.647969,0.027778,0.138889,5.704225,3.929577,1.140845,1.451613,1.563380,1.070423,151.750,17.098592,3.746914,-7.000000,-0.713136,0.000000,4.308172,0.173547,0.082561,0.270873,0.335177,0.423150,0.758326
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000,2020,4.434783,5.608696,4.985167,4.031506,0.645161,0.012945,0.152104,6.130435,2.347826,0.521739,2.611111,1.521739,0.942029,146.000,16.927536,3.779935,7.200000,-0.403277,1.173913,3.991623,0.198953,0.081588,0.266275,0.331132,0.397944,0.729076
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000,2020,2.957143,3.985714,3.882761,4.368152,0.736515,0.013158,0.144737,5.657143,3.728571,0.514286,1.517241,1.442857,1.057143,154.000,17.600000,4.039344,22.400000,1.411009,1.028571,4.392840,0.166269,0.082339,0.289097,0.352305,0.420912,0.773216
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857,2020,6.717073,7.112195,6.390847,5.733308,0.646018,0.030675,0.196319,8.429268,5.531707,1.317073,1.523810,1.770732,1.551220,163.875,19.185366,4.021472,-8.433333,-0.983765,0.395122,4.570111,0.176073,0.106916,0.271514,0.357772,0.408652,0.766424


F-E, R-E 수정

In [26]:
vs_team_pitcher['상대F-E'] = vs_team_pitcher['상대FIP'] - vs_team_pitcher['상대평균자책점']
vs_team_pitcher['상대R-E'] = vs_team_pitcher['상대RA/9'] - vs_team_pitcher['상대평균자책점']
team_pitcher['F-E'] = team_pitcher['FIP'] - team_pitcher['평균자책점']
team_pitcher['R-E'] = team_pitcher['RA/9'] - team_pitcher['평균자책점']

In [45]:
r_team_pitcher['최근F-E'] = r_team_pitcher['최근FIP'] - r_team_pitcher['최근평균자책점']
r_team_pitcher['최근R-E'] = r_team_pitcher['최근RA/9'] - r_team_pitcher['최근평균자책점']

공통 상대 데이터 구하기

In [32]:
team_data

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,결과,시즌,경기간간격,월,NC,OB,WO,HT,LG,SS,KT,LT,SK,HH,이닝*3,실점,우취,연장,득점,시작전총득점,시작전총실점,시작전평균득점,시작전평균실점,시작전승수,시작전패배수,시작전승률,피타고리안기대승률
0,20160401HHLG0,2016-04-01,LG,HH,0.0,0,W,2016.0,0.0,4.0,0,0,0,0,1,0,0,0,0,0,36,4,0,1,5,0,0,0.000000,0.000000,0,0,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0.0,0,L,2016.0,0.0,4.0,0,0,0,0,0,0,0,0,0,1,34,5,0,1,4,0,0,0.000000,0.000000,0,0,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0.0,0,W,2016.0,0.0,4.0,1,0,0,0,0,0,0,0,0,0,27,4,0,0,5,0,0,0.000000,0.000000,0,0,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0.0,0,L,2016.0,0.0,4.0,0,0,0,1,0,0,0,0,0,0,24,5,0,0,4,0,0,0.000000,0.000000,0,0,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0.0,0,L,2016.0,0.0,4.0,0,0,0,0,0,0,0,0,1,0,27,8,0,0,4,0,0,0.000000,0.000000,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,1,0,0,27,1,0,0,2,305,320,5.000000,5.245902,29,32,0.475410,0.476014
6396,20200719OBHT0,2020-07-19,HT,OB,0.0,0,L,2020.0,1.0,7.0,0,0,0,1,0,0,0,0,0,0,27,8,0,0,4,307,282,5.032787,4.622951,33,28,0.540984,0.542368
6397,20200719OBHT0,2020-07-19,OB,HT,0.0,0,W,2020.0,1.0,7.0,0,1,0,0,0,0,0,0,0,0,27,4,0,0,8,387,348,6.142857,5.523810,37,26,0.587302,0.552912
6398,20200719WOSK0,2020-07-19,SK,WO,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,0,1,0,27,3,0,0,4,248,338,3.875000,5.281250,20,44,0.312500,0.349956


In [33]:
# 함수 모아놓은 곳

def createInfo(row):
    previous = team_data[(team_data['팀코드'] == row['팀코드']) & (team_data['상대팀코드'] == row['상대팀코드'])& (team_data['일자'] < row['일자']) & (team_data['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['상대총득점'] = 0
        row['상대총실점'] = 0
        row['상대평균득점'] = 0
        row['상대평균실점'] = 0
    else:
        gainSum = previous['득점'].sum()
        gainMean = gainSum / len(previous)
        lossSum = previous['실점'].sum()
        lossMean = lossSum / len(previous)
        row['상대총득점'] = gainSum
        row['상대총실점'] = lossSum
        row['상대평균득점'] = gainMean
        row['상대평균실점'] = lossMean
    return row

def winP(row):
    previous = team_data[(team_data['팀코드'] == row['팀코드']) & (team_data['일자'] < row['일자']) & (team_data['상대팀코드'] == row['상대팀코드']) & (team_data['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['상대승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['상대승률'] = winMean
    return row

In [34]:
team_data = team_data.apply(createInfo, axis=1)
team_data = team_data.apply(winP, axis=1)
team_data['상대피타고리안기대승률'] = (team_data['상대총득점'] ** 2) / ((team_data['상대총득점'] ** 2) + (team_data['상대총실점'] ** 2))

In [36]:
team_data = team_data.fillna(0)

이동평균법

In [41]:
r_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig')
r_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig')

In [92]:
r_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_최근_팀투수.csv', encoding='utf-8-sig', index=False)
r_team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_최근_팀타자.csv', encoding='utf-8-sig', index=False)

In [51]:
def createInfo(row):
    previous = team_data[(team_data['팀코드'] == row['팀코드'])& (team_data['일자'] < row['일자']) & (team_data['시즌'] == row['시즌'])].copy().tail()
    if len(previous) == 0:
        row['최근총득점'] = 0
        row['최근총실점'] = 0
        row['최근평균득점'] = 0
        row['최근평균실점'] = 0
    else:
        gainSum = previous['득점'].sum()
        gainMean = gainSum / len(previous)
        lossSum = previous['실점'].sum()
        lossMean = lossSum / len(previous)
        row['최근총득점'] = gainSum
        row['최근총실점'] = lossSum
        row['최근평균득점'] = gainMean
        row['최근평균실점'] = lossMean
    return row

def winP(row):
    previous = team_data[(team_data['팀코드'] == row['팀코드']) & (team_data['일자'] < row['일자']) & (team_data['시즌'] == row['시즌'])].copy().tail()
    if len(previous) == 0:
        row['최근승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['최근승률'] = winMean
    return row

In [52]:
team_data = team_data.apply(createInfo, axis=1)
team_data = team_data.apply(winP, axis=1)
team_data['최근피타고리안기대승률'] = (team_data['최근총득점'] ** 2) / ((team_data['최근총득점'] ** 2) + (team_data['최근총실점'] ** 2))

In [67]:
def createSecA(row):
    previous = r_team_hitter[(r_team_hitter['시즌'] == row['시즌']) & (r_team_hitter['팀코드'] == row['팀코드']) & (r_team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['SecA'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['4구']+(previous_sum['2루타']+previous_sum['3루타']*2+previous_sum['홈런']*3)+previous_sum['도루']-previous_sum['도루실패']
    down = previous_sum['타수']
    if down == 0:
        row['SecA'] = 0
        return row
    row['SecA'] = up/down
    return row

def createPSN(row):
    previous = r_team_hitter[(r_team_hitter['시즌'] == row['시즌']) & (r_team_hitter['팀코드'] == row['팀코드']) & (r_team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['PSN'] = 0
        return row
    previous_sum = previous.sum()
    try:
        psn = 2*(previous_sum["홈런"]*previous_sum["도루"])/(previous_sum["홈런"]+previous_sum["도루"])
    except ZeroDivisionError:
        psn = 0
    row['PSN']= psn
    return row

In [68]:
r_team_hitter = r_team_hitter.apply(createSecA, axis=1)
r_team_hitter = r_team_hitter.apply(createPSN, axis=1)


In [70]:
r_team_hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,최근출루율,최근장타력,최근ops,최근타율,최근ISO,최근BABIP,최근BB/K,최근K%,최근BB%,최근wOBA,최근wRAA,최근wRC,최근HR%,최근GPA,최근RC,최근RC27,최근SPD,최근XR,최근득점,SecA,PSN
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.377551,0.471910,0.849461,0.320225,0.151685,0.354167,0.551724,0.146465,0.080808,0.383393,6.593049,21.658607,0.030303,0.287875,29.847191,6.151711,3.730115,29.278,7.0,0.247191,4.000000
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.351648,0.370370,0.722019,0.271605,0.098765,0.320312,0.548387,0.166667,0.086022,0.335436,-1.551068,12.677515,0.016129,0.250834,20.742724,4.341500,1.608126,20.232,3.2,0.197531,0.000000
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.375661,0.437870,0.813531,0.325444,0.112426,0.344156,0.764706,0.089005,0.068063,0.369455,4.051093,18.584030,0.010471,0.278515,26.727481,5.682220,5.779361,27.994,5.2,0.224852,3.200000
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.340102,0.375000,0.715102,0.278409,0.096591,0.318841,0.444444,0.182741,0.081218,0.329731,-2.608846,12.380623,0.025381,0.246796,24.189227,4.910595,5.286321,25.352,5.8,0.221591,5.833333


In [71]:
def createkwERA(row):
    previous = r_team_pitcher[(r_team_pitcher['시즌'] == row['시즌']) & (r_team_pitcher['팀코드'] == row['팀코드'])  & (r_team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['kwERA'] = 0
        return row
    previous_sum = previous.sum()
    try:
        up = 5.40-12*(previous_sum['삼진']-previous_sum['4구'])/previous_sum['타자']
    except ZeroDivisionError:
        row['kwERA'] = 0
        return row
    row['kwERA'] = up
    return row

def createK(row):
    previous = r_team_pitcher[(r_team_pitcher['시즌'] == row['시즌']) & (r_team_pitcher['팀코드'] == row['팀코드'])  & (r_team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['K%'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['삼진']
    down = previous_sum['타자']
    if down == 0:
        row['K%'] = 0
        return row
    row['K%'] = up/down
    return row

def createBB(row):
    previous = r_team_pitcher[(r_team_pitcher['시즌'] == row['시즌']) & (r_team_pitcher['팀코드'] == row['팀코드'])  & (r_team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['BB%'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['4구']
    down = previous_sum['타자']
    if down == 0:
        row['BB%'] = 0
        return row
    row['BB%'] = up/down
    return row

def createOAVG(row):
    previous = r_team_pitcher[(r_team_pitcher['시즌'] == row['시즌']) & (r_team_pitcher['팀코드'] == row['팀코드'])  & (r_team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['OAVG'] = 0
        return row
    previous_sum = previous.sum()
    try:
        oavg = previous_sum["안타"]/previous_sum["타수"]
    except ZeroDivisionError:
        oavg = 0
    row['OAVG']= oavg
    return row

def createOOBP(row):
    previous = r_team_pitcher[(r_team_pitcher['시즌'] == row['시즌']) & (r_team_pitcher['팀코드'] == row['팀코드'])  & (r_team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['OOBP'] = 0
        return row
    previous_sum = previous.sum()
    try:
        oobp = (previous_sum["안타"]+previous_sum["4구"]+previous_sum["사구"])/(previous_sum["타수"]+previous_sum["4구"]+previous_sum["사구"]+previous_sum["희비"])
    except ZeroDivisionError:
        oobp = 0
    row['OOBP']= oobp
    return row

def createOSLG(row):
    previous = r_team_pitcher[(r_team_pitcher['시즌'] == row['시즌']) & (r_team_pitcher['팀코드'] == row['팀코드'])  & (r_team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['OSLG'] = 0
        return row
    previous_sum = previous.sum()
    try:
        oslg = (previous_sum["단타"]+2*previous_sum["2루타"]+3*previous_sum["3루타"]+4*previous_sum["홈런"])/previous_sum["타수"]
    except ZeroDivisionError:
        oslg = 0
    row['OSLG']= oslg
    return row

In [72]:
try:
  r_team_pitcher = r_team_pitcher.apply(createkwERA, axis=1)
  r_team_pitcher = r_team_pitcher.apply(createK, axis=1)
  r_team_pitcher = r_team_pitcher.apply(createBB, axis=1)
  r_team_pitcher = r_team_pitcher.apply(createOAVG, axis=1)
  r_team_pitcher = r_team_pitcher.apply(createOOBP, axis=1)
  r_team_pitcher = r_team_pitcher.apply(createOSLG, axis=1)
  r_team_pitcher['OOPS'] = r_team_pitcher['OSLG'] + r_team_pitcher['OOBP']
except:
  r_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_최근_팀투수.csv', encoding='utf-8-sig', index=False)
  print('오류발생')
finally:
  r_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_최근_팀투수.csv', encoding='utf-8-sig', index=False)

오류발생


In [76]:
r_team_pitcher['단타'] = r_team_pitcher['안타'] - r_team_pitcher['2루타'] - r_team_pitcher['3루타'] - r_team_pitcher['홈런']

In [78]:
r_team_pitcher = r_team_pitcher.apply(createOSLG, axis=1)
r_team_pitcher['OOPS'] = r_team_pitcher['OSLG'] + r_team_pitcher['OOBP']

In [79]:
r_team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,최근평균자책점,최근RA/9,최근ERC,최근FIP,최근LOB,최근홈런율,최근삼진율,최근K9,최근BB9,최근HR9,최근KBB,최근whip,최근PFR,최근P/G,최근P/IP,최근P/PA,최근CYP,최근F-E,최근R-E,kwERA,K%,BB%,OAVG,OOBP,단타,OSLG,OOPS
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000,2020,8.162791,8.162791,6.980192,5.550211,0.578512,0.029412,0.132353,5.651163,3.767442,1.255814,1.500000,1.837209,1.069767,162.4,18.883721,3.960976,-12.800000,-2.612579,0.000000,4.931707,0.131707,0.092683,0.337079,0.400990,6,0.500000,0.900990
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000,2020,4.740458,4.740458,5.471856,5.589089,0.753846,0.026316,0.131579,5.152672,4.328244,1.030534,1.190476,1.580153,1.053435,150.4,17.221374,3.957895,3.333333,0.848631,0.000000,5.147368,0.131579,0.110526,0.303797,0.387097,7,0.424051,0.811147
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000,2020,3.477273,4.090909,4.394455,4.142178,0.748408,0.015544,0.170984,6.750000,3.068182,0.613636,2.200000,1.477273,1.113636,150.2,17.068182,3.871134,11.300000,0.664905,0.613636,4.348454,0.170103,0.082474,0.284884,0.348958,7,0.360465,0.709423
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857,2020,6.488372,6.488372,6.311318,5.340909,0.660661,0.030457,0.152284,6.279070,3.558140,1.255814,1.764706,1.697674,1.093023,158.2,18.395349,4.015228,-4.500000,-1.147463,0.000000,4.608122,0.152284,0.086294,0.318182,0.380711,6,0.500000,0.880711


최종 점검

In [80]:
team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀투수.csv', encoding='utf-8-sig')
pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인투수.csv', encoding='utf-8-sig')
team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀타자.csv', encoding='utf-8-sig')
hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인타자.csv', encoding='utf-8-sig')
team_data = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig')
vs_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀투수.csv', encoding='utf-8-sig')
vs_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀타자.csv', encoding='utf-8-sig')
r_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_최근_팀투수.csv', encoding='utf-8-sig')
r_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_최근_팀타자.csv', encoding='utf-8-sig')

In [28]:
team_data = pd.concat([team_data, team_pitcher[['총합연봉', '평균연봉']]], axis=1)
team_data

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,결과,시즌,경기간간격,월,NC,OB,WO,HT,LG,SS,KT,LT,SK,HH,이닝*3,실점,우취,연장,득점,시작전총득점,시작전총실점,시작전평균득점,시작전평균실점,시작전승수,시작전패배수,시작전승률,피타고리안기대승률,상대총득점,상대총실점,상대평균득점,상대평균실점,상대승률,상대피타고리안기대승률,최근총득점,최근총실점,최근평균득점,최근평균실점,최근승률,최근피타고리안기대승률,총합연봉,평균연봉
0,20160401HHLG0,2016-04-01,LG,HH,0.0,0,W,2016.0,0.0,4.0,0,0,0,0,1,0,0,0,0,0,36,4,0,1,5,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0.000,0.000,0.00,0.000000,0,0,0.0,0.0,0.0,0.000000,925700,14025.758
1,20160401HHLG0,2016-04-01,HH,LG,0.0,0,L,2016.0,0.0,4.0,0,0,0,0,0,0,0,0,0,1,34,5,0,1,4,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0.000,0.000,0.00,0.000000,0,0,0.0,0.0,0.0,0.000000,1454500,19655.405
2,20160401HTNC0,2016-04-01,NC,HT,0.0,0,W,2016.0,0.0,4.0,1,0,0,0,0,0,0,0,0,0,27,4,0,0,5,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0.000,0.000,0.00,0.000000,0,0,0.0,0.0,0.0,0.000000,872200,13017.910
3,20160401HTNC0,2016-04-01,HT,NC,0.0,0,L,2016.0,0.0,4.0,0,0,0,1,0,0,0,0,0,0,24,5,0,0,4,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0.000,0.000,0.00,0.000000,0,0,0.0,0.0,0.0,0.000000,1058700,15801.493
4,20160401KTSK0,2016-04-01,SK,KT,0.0,0,L,2016.0,0.0,4.0,0,0,0,0,0,0,0,0,1,0,27,8,0,0,4,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0.000,0.000,0.00,0.000000,0,0,0.0,0.0,0.0,0.000000,955500,14051.471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,1,0,0,27,1,0,0,2,305,320,5.000000,5.245902,29,32,0.475410,0.476014,27,49,3.375,6.125,0.25,0.232907,35,39,7.0,7.8,0.4,0.446103,1067700,16682.812
6396,20200719OBHT0,2020-07-19,HT,OB,0.0,0,L,2020.0,1.0,7.0,0,0,0,1,0,0,0,0,0,0,27,8,0,0,4,307,282,5.032787,4.622951,33,28,0.540984,0.542368,31,43,3.875,5.375,0.25,0.341993,16,23,3.2,4.6,0.4,0.326115,998090,15842.698
6397,20200719OBHT0,2020-07-19,OB,HT,0.0,0,W,2020.0,1.0,7.0,0,1,0,0,0,0,0,0,0,0,27,4,0,0,8,387,348,6.142857,5.523810,37,26,0.587302,0.552912,43,31,5.375,3.875,0.75,0.658007,26,20,5.2,4.0,0.6,0.628253,975000,15476.190
6398,20200719WOSK0,2020-07-19,SK,WO,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,0,1,0,27,3,0,0,4,248,338,3.875000,5.281250,20,44,0.312500,0.349956,38,54,4.750,6.750,0.25,0.331193,29,31,5.8,6.2,0.4,0.466704,935800,15093.548


In [30]:
team_pitcher = team_pitcher[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '평균자책점', 'RA/9', 'ERC', 'FIP', 'kwERA','LOB',
       '홈런율','K%', 'BB%', 'K9', 'BB9', 'HR9', 'KBB', 'whip', 'P/G', 'P/IP', 'P/PA',
       'PFR', 'CYP', 'F-E', 'R-E', 'OAVG', 'OOBP', 
       'OSLG', 'OOPS']]



In [34]:
team_pitcher.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '평균자책점', 'RA/9', 'ERC', 'FIP', 'kwERA',
       'LOB', 'HR%', 'K%', 'BB%', 'K9', 'BB9', 'HR9', 'KBB', 'whip', 'P/G',
       'P/IP', 'P/PA', 'PFR', 'CYP', 'F-E', 'R-E', 'OAVG', 'OOBP', 'OSLG',
       'OOPS']

In [39]:
team_hitter = team_hitter[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수', '타점', '득점',
       '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구', '고4', '사구',
       '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌', '단타', '출루율',
       '장타력', 'ops', 'GPA', '타율', 'BABIP', 'ISO', 'HR%', 'K%', 'BB%','BB/K','wOBA', 'wRAA',
       'wRC', 'RC', 'RC27', 'XR', 'SPD', 'SecA',
       'PSN']]

In [41]:
team_hitter.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수', '타점', '득점',
       '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구', '고4', '사구',
       '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌', '단타', 'OBP',
       'SLG', 'OPS', 'GPA', '타율', 'BABIP', 'ISO', 'HR%', 'K%', 'BB%', 'BBK',
       'wOBA', 'wRAA', 'wRC', 'RC', 'RC27', 'XR', 'SPD', 'SECA', 'PSN']

In [46]:
pitcher= pitcher[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '선수코드', '선발', '구원', '완투',
       '종료', '결과', '홀드', '이닝*3', '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런',
       '도루', '도루.1', '희타', '희비', '4구', '고4', '사구', '삼진', '병살', '폭투', '보크',
       '실책', '실점', '자책점', '득점권WHIP', '2점차이하WHIP', '345번타자WHIP', '시즌', '평균자책점',
       'RA/9', 'ERC', 'FIP', 'kwERA', 'LOB', '홈런율', 'perK%', 'perBB%','K9', 'BB9', 'HR9', 'KBB',
       'whip', 'P/G', 'P/IP', 'P/PA', 'PFR', 'CYP', 'F-E', 'R-E', '연봉', 'OAVG',
       'OOBP', 'OSLG', 'OOPS']]
pitcher.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '선수코드', '선발', '구원', '완투',
       '종료', '결과', '홀드', '이닝*3', '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런',
       '도루', '도루실패', '희타', '희비', '4구', '고4', '사구', '삼진', '병살', '폭투', '보크',
       '실책', '실점', '자책점', '득점권WHIP', '2점차이하WHIP', '345번타자WHIP', '시즌', '평균자책점',
       'RA/9', 'ERC', 'FIP', 'kwERA', 'LOB', 'HR%', 'K%', 'perBB%','K9', 'BB9', 'HR9', 'KBB',
       'whip', 'P/G', 'P/IP', 'P/PA', 'PFR', 'CYP', 'F-E', 'R-E', '연봉', 'OAVG',
       'OOBP', 'OSLG', 'OOPS']

In [51]:
hitter = hitter[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '선수코드', '선발', '타순', '타자',
       '타수', '타점', '득점', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타',
       '시즌', '단타', '출루율', '장타력', 'ops', 'GPA', '타율', 'BABIP',  'ISO','HR%', 'K%',
       'BB%', 'BB/K', 'wOBA', 'wRAA', 'wRC', 'RC', 'XR', 'RC27', 'SPD',
       'SecA', 'PSN', '연봉']]

hitter.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '선수코드', '선발', '타순', '타자',
       '타수', '타점', '득점', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타',
       '시즌', '단타', 'OBP', 'SLG', 'OPS', 'GPA', '타율', 'BABIP',  'ISO','HR%', 'K%',
       'BB%', 'BB/K', 'wOBA', 'wRAA', 'wRC', 'RC', 'XR', 'RC27', 'SPD',
       'SECA', 'PSN', '연봉']

In [55]:
team_data.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '결과', '시즌', '경기간간격', '월',
       'NC', 'OB', 'WO', 'HT', 'LG', 'SS', 'KT', 'LT', 'SK', 'HH', '이닝*3',
       '실점', '우취', '연장', '득점', '총득점', '총실점', '평균득점', '평균실점',
       '승수', '패배수', '승률', '피타고리안기대승률', '상대총득점', '상대총실점', '상대평균득점',
       '상대평균실점', '상대승률', '상대피타고리안기대승률', '최근총득점', '최근총실점', '최근평균득점', '최근평균실점',
       '최근승률', '최근피타고리안기대승률', '총합연봉', '평균연봉']

In [72]:
vs_team_pitcher = vs_team_pitcher[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '상대평균자책점', '상대RA/9', '상대ERC', '상대FIP','kwERA',
       '상대LOB', '상대홈런율', 'K%',
       'BB%', '상대K9', '상대BB9', '상대HR9', '상대KBB', '상대whip',
       'PFR', 'P/G', 'P/IP', 'P/PA', 'CYP', '상대F-E', '상대R-E', 'OAVG', 'OOBP', 'OSLG', 'OOPS']]
vs_team_pitcher.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '상대평균자책점', '상대RA/9', '상대ERC', '상대FIP','상대kwERA',
       '상대LOB', '상대HR%', '상대K%',
       '상대BB%', '상대K9', '상대BB9', '상대HR9', '상대KBB', '상대whip',
       '상대PFR', '상대P/G', '상대P/IP', '상대P/PA', '상대CYP', '상대F-E', '상대R-E', '상대OAVG', '상대OOBP', '상대OSLG', '상대OOPS']

In [73]:
vs_team_hitter = vs_team_hitter[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수',
       '타점', '득점', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구',
       '고4', '사구', '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌',
       '단타', '상대출루율', '상대장타력', '상대ops', '상대GPA', '상대타율', '상대ISO', '상대BABIP', 
       '상대HR%', '상대K%', '상대BB%','상대BB/K', '상대wOBA', '상대wRAA', '상대wRC', 'RC', 'RC27', 'XR', 'SPD', 'SecA', 'PSN']]

vs_team_hitter.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수',
       '타점', '득점', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구',
       '고4', '사구', '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌',
       '단타', '상대OBP', '상대SLG', '상대OPS', '상대GPA', '상대타율', '상대ISO', '상대BABIP', 
       '상대HR%', '상대K%', '상대BB%','상대BBK', '상대wOBA', '상대wRAA', '상대wRC', '상대RC',
       '상대RC27', '상대XR', '상대SPD', '상대SECA', '상대PSN']

In [81]:
r_team_hitter = r_team_hitter[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수', '타점', '득점',
       '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구', '고4', '사구',
       '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌', '단타', '최근출루율',
       '최근장타력', '최근ops','최근GPA', '최근타율', '최근ISO', '최근BABIP', '최근HR%', '최근K%', '최근BB%', '최근BB/K',
       '최근wOBA', '최근wRAA', '최근wRC',   '최근RC', '최근RC27','최근XR',
       '최근SPD', 'SecA', 'PSN']]

r_team_hitter.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수', '타점', '득점',
       '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구', '고4', '사구',
       '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌', '단타', '최근OBP',
       '최근SLG', '최근OPS','최근GPA', '최근타율', '최근ISO', '최근BABIP', '최근HR%', '최근K%', '최근BB%', '최근BBK',
       '최근wOBA', '최근wRAA', '최근wRC', '최근RC', '최근RC27','최근XR',
       '최근SPD', '최근SECA', '최근PSN']


In [86]:
r_team_pitcher = r_team_pitcher[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '최근평균자책점', '최근RA/9', '최근ERC', '최근FIP','kwERA',
       '최근LOB', '최근홈런율', 'K%', 'BB%', '최근K9', '최근BB9', '최근HR9', '최근KBB', '최근whip',
       '최근PFR', '최근P/G', '최근P/IP', '최근P/PA', '최근CYP', '최근F-E', '최근R-E',
      'OAVG', 'OOBP', 'OSLG', 'OOPS']]

r_team_pitcher.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '최근평균자책점', '최근RA/9', '최근ERC', '최근FIP','최근kwERA',
       '최근LOB', '최근HR%', '최근K%', '최근BB%', '최근K9', '최근BB9', '최근HR9', '최근KBB', '최근whip',
       '최근PFR', '최근P/G', '최근P/IP', '최근P/PA', '최근CYP', '최근F-E', '최근R-E',
      '최근OAVG', '최근OOBP', '최근OSLG', '최근OOPS']

In [87]:
r_team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,최근평균자책점,최근RA/9,최근ERC,최근FIP,최근kwERA,최근LOB,최근HR%,최근K%,최근BB%,최근K9,최근BB9,최근HR9,최근KBB,최근whip,최근PFR,최근P/G,최근P/IP,최근P/PA,최근CYP,최근F-E,최근R-E,최근OAVG,최근OOBP,최근OSLG,최근OOPS
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000,2020,8.162791,8.162791,6.980192,5.550211,4.931707,0.578512,0.029412,0.131707,0.092683,5.651163,3.767442,1.255814,1.500000,1.837209,1.069767,162.4,18.883721,3.960976,-12.800000,-2.612579,0.000000,0.337079,0.400990,0.500000,0.900990
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000,2020,4.740458,4.740458,5.471856,5.589089,5.147368,0.753846,0.026316,0.131579,0.110526,5.152672,4.328244,1.030534,1.190476,1.580153,1.053435,150.4,17.221374,3.957895,3.333333,0.848631,0.000000,0.303797,0.387097,0.424051,0.811147
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000,2020,3.477273,4.090909,4.394455,4.142178,4.348454,0.748408,0.015544,0.170103,0.082474,6.750000,3.068182,0.613636,2.200000,1.477273,1.113636,150.2,17.068182,3.871134,11.300000,0.664905,0.613636,0.284884,0.348958,0.360465,0.709423
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857,2020,6.488372,6.488372,6.311318,5.340909,4.608122,0.660661,0.030457,0.152284,0.086294,6.279070,3.558140,1.255814,1.764706,1.697674,1.093023,158.2,18.395349,4.015228,-4.500000,-1.147463,0.000000,0.318182,0.380711,0.500000,0.880711


In [90]:
team_data = team_data.round(3)
vs_team_pitcher = vs_team_pitcher.round(3)
r_team_pitcher = r_team_pitcher.round(3)
r_team_hitter = r_team_hitter.round(3)
vs_team_hitter = vs_team_hitter.round(3)


In [91]:
vs_team_hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,상대OBP,상대SLG,상대OPS,상대GPA,상대타율,상대ISO,상대BABIP,상대HR%,상대K%,상대BB%,상대BBK,상대wOBA,상대wRAA,상대wRC,상대RC,상대RC27,상대XR,상대SPD,상대SECA,상대PSN
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333,12,4,2016,6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200,15,3,2016,11,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.143,7,1,2016,5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100,10,1,2016,4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375,8,3,2016,4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000,2,0,2020,4,0.300,0.315,0.614,0.214,0.228,0.086,0.264,0.010,0.154,0.087,0.565,0.288,-14.656,8.018,25.755,3.249,27.252,4.449,0.206,4.200
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571,7,4,2020,7,0.340,0.390,0.730,0.250,0.266,0.124,0.303,0.013,0.145,0.095,0.682,0.334,-2.931,20.276,34.422,4.405,34.698,3.027,0.232,1.600
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.364,11,4,2020,7,0.364,0.433,0.797,0.272,0.314,0.119,0.361,0.013,0.152,0.058,0.383,0.363,4.736,28.247,40.376,5.167,40.444,3.683,0.199,4.444
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250,8,2,2020,6,0.307,0.366,0.673,0.230,0.246,0.120,0.287,0.026,0.200,0.075,0.377,0.308,-9.805,13.402,30.932,3.796,31.132,3.919,0.199,1.778
